In [234]:
import pyscf

In [235]:
molecule = """
Cr -1.00 0.00 -0.00
Cr 1.0 0.00 -0.00
"""

In [236]:
basis = "def2-svp"
pymol = pyscf.gto.Mole(
        atom    =   molecule,
        symmetry=   True,
        spin    =   0,
        charge  =   0,
        basis   =   basis)


pymol.build()
print("symmetry: ",pymol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(pymol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
mf.analyze()

symmetry:  Dooh


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 253 MB)
num. doubly occ = 24  num. singly occ = 0
init E= -2084.89003163867
HOMO (E2gx) = -0.271939990572288  LUMO (E2uy) = -0.228365993540994
cycle= 1 E= -2085.27675222348  delta_E= -0.387  |g|= 0.63  |ddm|= 2.15
HOMO (E2gy) = -0.0661020165770354  LUMO (A1u) = 0.0206737172478062
cycle= 2 E= -2085.34156118261  delta_E= -0.0648  |g|= 0.465  |ddm|= 0.906
HOMO (A1g) = -0.202007607931212  LUMO (A1u) = 0.00610350541105471
cycle= 3 E= -2085.38797010495  delta_E= -0.0464  |g|= 0.0336  |ddm|= 0.261
HOMO (A1g) = -0.206746661240235  LUMO (A1u) = 0.00330741814584089
cycle= 4 E=

((array([1.99999949e+00, 1.99999588e+00, 1.99995470e+00, 9.97779600e-01,
         6.65564394e-03, 1.99999551e+00, 1.99999906e+00, 1.99999906e+00,
         1.99952808e+00, 1.99961641e+00, 1.99961641e+00, 1.61696632e-04,
         3.56691936e-02, 3.56691936e-02, 9.60781255e-01, 9.95976966e-01,
         9.95753776e-01, 9.60781255e-01, 9.95307396e-01, 3.70938598e-03,
         3.98524669e-03, 3.12529597e-03, 3.70938598e-03, 1.40539452e-03,
         2.10657308e-04, 3.77871771e-05, 1.40438206e-05, 8.42629234e-05,
         1.36027991e-04, 1.40438206e-04, 2.01521866e-04, 1.99999949e+00,
         1.99999588e+00, 1.99995470e+00, 9.97779600e-01, 6.65564394e-03,
         1.99999551e+00, 1.99999906e+00, 1.99999906e+00, 1.99952808e+00,
         1.99961641e+00, 1.99961641e+00, 1.61696632e-04, 3.56691936e-02,
         3.56691936e-02, 9.60781255e-01, 9.95976966e-01, 9.95753776e-01,
         9.60781255e-01, 9.95307396e-01, 3.70938598e-03, 3.98524669e-03,
         3.12529597e-03, 3.70938598e-03, 1.40539452

In [237]:
F = mf.get_fock()
import numpy as np
import scipy
import copy as cp
import math

In [238]:
def sym_ortho(frags, S, thresh=1e-8):
    Nbas = S.shape[1]
    
    inds = []
    Cnonorth = np.hstack(frags)
    shift = 0
    for f in frags:
        inds.append(list(range(shift, shift+f.shape[1])))
        shift += f.shape[1]
        
    
    Smo = Cnonorth.T @ S @ Cnonorth
    X = np.linalg.inv(scipy.linalg.sqrtm(Smo))
    # print(Cnonorth.shape, X.shape)
    Corth = Cnonorth @ X
    
    frags2 = []
    for f in inds:
        frags2.append(Corth[:,f])
    return frags2

def semi_canonicalize(C,F):
    e,V = np.linalg.eigh(C.T @ F @ C)
    for ei in e:
        print(" Orbital Energy: ", ei)
    return C @ V
import scipy


In [239]:
# Find AO's corresponding to atoms 
# print(mf.mol.aoslice_by_atom())
# print(mf.mol.ao_labels(fmt=False, base=0))
full = []
frag1 = []
frag2 = []
for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        if ao[2] in ("3d", "4s"):
            frag1.append(ao_idx)
            full.append(ao_idx)
    elif ao[0] == 1:
        if ao[2] in ("3d", "4s"):
            frag2.append(ao_idx)
            full.append(ao_idx)
total=[]

for ao_idx,ao in enumerate(mf.mol.ao_labels(fmt=False)):
    if ao[0] == 0:
        total.append(ao_idx)
    elif ao[0] == 1:
        total.append(ao_idx)
print(total)
for i in full:
    del total[i]
env=[]
env=total
print(env)
print(frag1)
print(frag2)
frags = [frag1, frag2]
P = mf.make_rdm1()
Pa = P[0,:,:]
Pb = P[1,:,:]

C = mf.mo_coeff
S = mf.get_ovlp()

print(full)
cf1=C[:,frag1]
cf2=C[:,frag2]
C1, C2 = sym_ortho([cf1, cf2], S)
C1 = semi_canonicalize(C1, F)
C2 = semi_canonicalize(C2, F)

Cact = np.hstack((C1, C2))
#print(cf1)
print(C1)
Cact = np.hstack((C1, C2))
Cenv=C[:,env]

pyscf.tools.molden.from_mo(mf.mol, "Cact.molden", Cact);
clusters=[frag1,frag2]
init_fspace=[(5,5),(5,5)]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 20, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 55, 57, 59, 61]
[3, 14, 15, 16, 17, 18]
[34, 45, 46, 47, 48, 49]
[3, 14, 15, 16, 17, 18, 34, 45, 46, 47, 48, 49]
 Orbital Energy:  -26.521622140799437
 Orbital Energy:  -2.243206775701061
 Orbital Energy:  -2.222054516291731
 Orbital Energy:  -2.2220545162917245
 Orbital Energy:  -2.199161403299623
 Orbital Energy:  -0.317108472238779
 Orbital Energy:  0.2294239057262734
 Orbital Energy:  1.0110655095695937
 Orbital Energy:  1.0320646453228393
 Orbital Energy:  1.0320646453228441
 Orbital Energy:  3.1352189259303596
 Orbital Energy:  3.1941536385255165
[[-2.34341327e-01  1.01849

In [240]:
print(clusters)
print(init_fspace)

[[3, 14, 15, 16, 17, 18], [34, 45, 46, 47, 48, 49]]
[(5, 5), (5, 5)]


In [241]:

d1_embed = 2 * Cenv @ Cenv.T

h0 = pyscf.gto.mole.energy_nuc(mf.mol)
h  = pyscf.scf.hf.get_hcore(mf.mol)
j, k = pyscf.scf.hf.get_jk(mf.mol, d1_embed, hermi=1)
h0 += np.trace(d1_embed @ ( h + .5*j - .25*k))

h = Cact.T @ h @ Cact;
j = Cact.T @ j @ Cact;
k = Cact.T @ k @ Cact;
nact = h.shape[0]

h2 = pyscf.ao2mo.kernel(pymol, Cact, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

In [242]:
S = pymol.intor("int1e_ovlp_sph")
n_act = np.trace(S @ d1_embed @ S @ Cact @ Cact.T)
if abs(n_act) > 1e-8 == False:
    print(n_act)
    error(" I found embedded electrons in the active space?!")

h1 = h + j - .5*k;
np.save("ints_h0", h0)
np.save("ints_h1", h1)
np.save("ints_h2", h2)
np.save("mo_coeffs", Cact)
np.save("overlap_mat", S)
Cold = mf.mo_coeff
U = Cold.T @ S @ Cact
Pa = mf.make_rdm1()[0]
Pb = mf.make_rdm1()[1]

Pa = Cact.T @ S @ Pa @ S @ Cact
Pb = Cact.T @ S @ Pb @ S @ Cact
print(np.trace(Pa), np.trace(Pb))
np.save("Pa", Pa)
np.save("Pb", Pb)
np.trace(Cact @ Cact.T @ S @ F) 

6.000000000000002 6.000000000000002


-29.499637670573016